In [530]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [531]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

workers - workers.csv


,worker_id,name,плоская,высокая,струйная
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


equipment - equipment.csv


,machine_id,machine_type,operators_needed
0,PM-01,плоская,4
1,PM-02,плоская,4
2,PM-03,высокая,4
3,PM-04,высокая,4
4,SM-01,струйная,1


schedule - schedule_template.csv


,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


requirements - position_requirements.csv


,machine_type,position,min_rank,profession_required
0,плоская,1,7,Печатник
1,плоская,2,6,Печатник
2,плоская,3,5,Печатник
3,плоская,4,4,Печатник
4,струйная,1,5,Оператор струйной печати


plan - plan.csv


,machine_id,night,day,evening
0,PM-01,True,True,True
1,PM-02,True,True,True
2,PM-03,True,True,True
3,PM-04,True,True,True
4,SM-01,True,True,True


In [532]:
# Определяем основную профессию
cols = ["плоская", "высокая", "струйная"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,W001,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,W002,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,W003,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,W004,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,W005,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [533]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
target_shift = "День"
shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}

In [534]:
# Функция для переворота смены
def shift_next_week(week, shift_name):
    shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}
    print(f"shift_workers - Кандидаты для работы в смену: {shift_name} недели {week}")
    result = schedule[
        (schedule["week"] == week - 1)
        &
        # (schedule["day"] == target_day) &
        (schedule["shift"] == shift[shift_name])
    ]
    result = result.merge(workers, on="worker_id", how="left")
    print(f"В смене работает {len(result)} человек")

    return result

In [535]:
shift_workers_day = shift_next_week(target_week, "День")
display(shift_workers_day.head())

shift_workers_evning = shift_next_week(target_week, "Вечер")
# display(shift_workers_evning)

shift_workers_nigth = shift_next_week(target_week, "Ночь")
# display(shift_workers_nigth)

shift_workers - Кандидаты для работы в смену: День недели 2
В смене работает 20 человек


,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,1,Вечер,W021,Работник П 21,7,6,0,плоская,"[плоская, высокая]"
1,1,Вечер,W022,Работник П 22,6,5,0,плоская,"[плоская, высокая]"
2,1,Вечер,W023,Работник П 23,5,4,0,плоская,"[плоская, высокая]"
3,1,Вечер,W024,Работник П 24,4,0,5,струйная,"[плоская, струйная]"
4,1,Вечер,W025,Работник П 25,7,6,0,плоская,"[плоская, высокая]"


shift_workers - Кандидаты для работы в смену: Вечер недели 2
В смене работает 20 человек
shift_workers - Кандидаты для работы в смену: Ночь недели 2
В смене работает 20 человек


In [536]:
def build_shift_rotation(schedule: pd.DataFrame, workers: pd.DataFrame, target_week: int) -> pd.DataFrame:
    """
    Формирует общий датафрейм кандидатов на target_week для всех смен сразу,
    применяя правило переворота смен:
        Ночь -> День, День -> Вечер, Вечер -> Ночь
    """
    shift_map = {"Ночь": "Вечер", "День": "Ночь", "Вечер": "День"}

    # Базовый слой — прошлая неделя
    prev = schedule.loc[schedule["week"] == target_week - 1].copy()

    # Сохраним прошлую смену (на всякий случай для анализа)
    prev = prev.rename(columns={"shift": "prev_shift"})

    # Рассчитаем смену на следующую неделю и проставим неделю
    prev["shift"] = prev["prev_shift"].map(shift_map)
    prev["week"] = target_week

    # Объединим с данными по работникам
    result = prev.merge(workers, on="worker_id", how="left")

    # Короткая статистика по сменам
    counts = result["shift"].value_counts()
    total = int(counts.sum())
    print(f"Кандидаты на работу в неделю {target_week}: всего {total}")
    for s in ["День", "Вечер", "Ночь"]:
        print(f"  {s}: {int(counts.get(s, 0))}")

    return result


In [537]:
shift_workers_all = build_shift_rotation(schedule, workers, target_week)
display(shift_workers_all.head())

# # При необходимости — получить конкретную смену:
# shift_workers_day   = shift_workers_all.query('shift == "День"')
# shift_workers_even  = shift_workers_all.query('shift == "Вечер"')
# shift_workers_night = shift_workers_all.query('shift == "Ночь"')


Кандидаты на работу в неделю 2: всего 60
  День: 20
  Вечер: 20
  Ночь: 20


,week,prev_shift,worker_id,shift,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,2,Ночь,W001,Вечер,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,2,Ночь,W002,Вечер,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,2,Ночь,W003,Вечер,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,2,Ночь,W004,Вечер,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,2,Ночь,W005,Вечер,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [538]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None
    
    return shift_equipment

In [539]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# Заменяем названия смен на русские
plan_long = plan_long.replace(
    {"shift": {"night": "Ночь", "day": "День", "evening": "Вечер"}}
)

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week

# Позже убрать дублирование кода
shift_equipment = f_shift_equipment(plan_long, target_shift)

# Создаем расписание для каждой смены с пустыми позициями работников
shift_equipment_day = f_shift_equipment(plan_long, 'День')
shift_equipment_evening = f_shift_equipment(plan_long, 'Вечер')
shift_equipment_night = f_shift_equipment(plan_long, 'Ночь')


display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,None
1,2,День,PM-01,плоская,2,6,Печатник,None
2,2,День,PM-01,плоская,3,5,Печатник,None
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,None
1,2,Вечер,PM-01,плоская,2,6,Печатник,None
2,2,Вечер,PM-01,плоская,3,5,Печатник,None
3,2,Вечер,PM-01,плоская,4,4,Печатник,None
4,2,Вечер,PM-02,плоская,1,7,Печатник,None


In [540]:
def candidat(s, profesion, min_rank, shift_name):

    if s == 'ferst':
        candidates = shift_workers_all[
        (shift_workers_all["shift"] == shift_name)
        & (shift_workers_all["основная_профессия"] == profesion)
        & (shift_workers_all[profesion] == min_rank)
        & (~shift_workers_all["worker_id"].isin(appointed))
            ]
    elif s == 'second':
        candidates = shift_workers_all[
        (shift_workers_all["shift"] == shift_name)
        & (shift_workers_all["все_профессии"].apply(lambda x: profesion in x))
        & (shift_workers_all[profesion] == min_rank)
        & (~shift_workers_all["worker_id"].isin(appointed))
            ]
    return candidates



def position_assignment(shift_equipment, s='ferst', shift_name = 'День'):
    # appointed = set()
    free_machin_positions = []
    
    for index, row in shift_equipment.iterrows():
        if not row["worker_id"]:
            print(row["worker_id"], "->", end=" ")
            profesion = row["machine_type"]
            min_rank = row["min_rank"]

            candidates = candidat(s, profesion, min_rank, shift_name)

            # candidates = shift_workers_day[
            #     (shift_workers_day["основная_профессия"] == profesion)
            #     & (shift_workers_day[profesion] == min_rank)
            #     & (~shift_workers_day["worker_id"].isin(appointed))
            # ]
            # print(candidates)

            if not candidates.empty:
                first = candidates.iloc[0]  # <-- первая строка
                worker_id = first["worker_id"]

                shift_equipment.loc[index, "worker_id"] = worker_id  # <-- loc, не iloc
                appointed.add(worker_id)
                print(worker_id)
            else:
                # при необходимости логировать отсутствие кандидатов
                print(f"Нет кандидатов для index={index}, {profesion}, min_rank={min_rank}")
                free_machin_positions.append(shift_equipment.loc[index])
        else:
            # print(row["worker_id"], "уже назначен")
            pass
    
        
    return pd.DataFrame(free_machin_positions), shift_equipment

appointed = set()

In [541]:
def run_assignment_for_shift(shift_equipment, default_tourse):
    # Первый тур назначения - основная профессия, минимальный ранг
    free_machin_positions, shift_equipment = position_assignment(shift_equipment, default_tourse[0][0], default_tourse[0][1])

    if not free_machin_positions.empty:
        print(f"Остались свободные позиции оборудования после тура 1 назначения:")
        for i in range(5):
            profession, shift_name = default_tourse[i + 1]
            # print(f"Остались свободные позиции оборудования после тура {i + 1} назначения:")
            # display(free_machin_positions)
            free_machin_positions, shift_equ = position_assignment(free_machin_positions, profession, shift_name)
            shift_equipment = shift_equipment.combine_first(shift_equ)
            if free_machin_positions.empty:
                break
            print(f"Остались свободные позиции оборудования после тура {i + 2} назначения:")
            display(free_machin_positions)
    return shift_equipment

In [542]:
default_tourse = [
    ("ferst",  "День"),
    ("second", "День"),
    ("ferst",  "Ночь"),
    ("second", "Ночь"),
    ("ferst",  "Вечер"),
    ("second", "Вечер"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

shift_equipment_day = run_assignment_for_shift(shift_equipment_day, default_tourse_day)
shift_equipment_evening = run_assignment_for_shift(shift_equipment_evening, default_tourse_evening)
shift_equipment_night = run_assignment_for_shift(shift_equipment_night, default_tourse_night)

# shift_equipment_day

None -> W021
None -> W022
None -> W023
None -> Нет кандидатов для index=3, плоская, min_rank=4
None -> W025
None -> W026
None -> W027
None -> Нет кандидатов для index=7, плоская, min_rank=4
None -> W029
None -> W030
None -> W031
None -> Нет кандидатов для index=11, высокая, min_rank=4
None -> W033
None -> W034
None -> W035
None -> Нет кандидатов для index=15, высокая, min_rank=4
None -> W024
None -> W028
None -> W032
None -> W036
None -> Нет кандидатов для index=20, струйная, min_rank=5
None -> Нет кандидатов для index=21, струйная, min_rank=5
None -> Нет кандидатов для index=22, струйная, min_rank=5
None -> Нет кандидатов для index=23, струйная, min_rank=5
None -> Нет кандидатов для index=24, струйная, min_rank=5
None -> Нет кандидатов для index=25, струйная, min_rank=5
None -> Нет кандидатов для index=26, струйная, min_rank=5
None -> Нет кандидатов для index=27, струйная, min_rank=5
None -> Нет кандидатов для index=28, струйная, min_rank=5
None -> Нет кандидатов для index=29, струйна

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
3,2,День,PM-01,плоская,4,4,Печатник,None
7,2,День,PM-02,плоская,4,4,Печатник,None
11,2,День,PM-03,высокая,4,4,Печатник,None
15,2,День,PM-04,высокая,4,4,Печатник,None
24,2,День,SM-09,струйная,1,5,Оператор струйной печати,None
25,2,День,SM-10,струйная,1,5,Оператор струйной печати,None
26,2,День,SM-11,струйная,1,5,Оператор струйной печати,None
27,2,День,SM-12,струйная,1,5,Оператор струйной печати,None
28,2,День,SM-13,струйная,1,5,Оператор струйной печати,None
29,2,День,SM-14,струйная,1,5,Оператор струйной печати,None


None -> Нет кандидатов для index=3, плоская, min_rank=4
None -> Нет кандидатов для index=7, плоская, min_rank=4
None -> Нет кандидатов для index=11, высокая, min_rank=4
None -> Нет кандидатов для index=15, высокая, min_rank=4
None -> W044
None -> W048
None -> W052
None -> W056
None -> Нет кандидатов для index=28, струйная, min_rank=5
None -> Нет кандидатов для index=29, струйная, min_rank=5
None -> Нет кандидатов для index=30, струйная, min_rank=5
None -> Нет кандидатов для index=31, струйная, min_rank=5
None -> Нет кандидатов для index=32, струйная, min_rank=5
None -> Нет кандидатов для index=33, струйная, min_rank=5
None -> Нет кандидатов для index=34, струйная, min_rank=5
None -> Нет кандидатов для index=35, струйная, min_rank=5
None -> Нет кандидатов для index=36, струйная, min_rank=5
Остались свободные позиции оборудования после тура 3 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
3,2,День,PM-01,плоская,4,4,Печатник,None
7,2,День,PM-02,плоская,4,4,Печатник,None
11,2,День,PM-03,высокая,4,4,Печатник,None
15,2,День,PM-04,высокая,4,4,Печатник,None
28,2,День,SM-13,струйная,1,5,Оператор струйной печати,None
29,2,День,SM-14,струйная,1,5,Оператор струйной печати,None
30,2,День,SM-15,струйная,1,5,Оператор струйной печати,None
31,2,День,SM-16,струйная,1,5,Оператор струйной печати,None
32,2,День,SM-17,струйная,1,5,Оператор струйной печати,None
33,2,День,SM-18,струйная,1,5,Оператор струйной печати,None


None -> W051
None -> W055
None -> W043
None -> W047
None -> W057
None -> W058
None -> W059
None -> W060
None -> Нет кандидатов для index=32, струйная, min_rank=5
None -> Нет кандидатов для index=33, струйная, min_rank=5
None -> Нет кандидатов для index=34, струйная, min_rank=5
None -> Нет кандидатов для index=35, струйная, min_rank=5
None -> Нет кандидатов для index=36, струйная, min_rank=5
Остались свободные позиции оборудования после тура 4 назначения:


...

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
10,2,Ночь,PM-03,высокая,3,5,Печатник,None
11,2,Ночь,PM-03,высокая,4,4,Печатник,None
14,2,Ночь,PM-04,высокая,3,5,Печатник,None
15,2,Ночь,PM-04,высокая,4,4,Печатник,None
16,2,Ночь,SM-01,струйная,1,5,Оператор струйной печати,None


None -> Нет кандидатов для index=2, плоская, min_rank=5
None -> Нет кандидатов для index=3, плоская, min_rank=4
None -> Нет кандидатов для index=4, плоская, min_rank=7
None -> Нет кандидатов для index=6, плоская, min_rank=5
None -> Нет кандидатов для index=7, плоская, min_rank=4
None -> Нет кандидатов для index=10, высокая, min_rank=5
None -> Нет кандидатов для index=11, высокая, min_rank=4
None -> Нет кандидатов для index=14, высокая, min_rank=5
None -> Нет кандидатов для index=15, высокая, min_rank=4
None -> Нет кандидатов для index=16, струйная, min_rank=5
None -> Нет кандидатов для index=17, струйная, min_rank=5
None -> Нет кандидатов для index=18, струйная, min_rank=5
None -> Нет кандидатов для index=19, струйная, min_rank=5
Остались свободные позиции оборудования после тура 5 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
10,2,Ночь,PM-03,высокая,3,5,Печатник,None
11,2,Ночь,PM-03,высокая,4,4,Печатник,None
14,2,Ночь,PM-04,высокая,3,5,Печатник,None
15,2,Ночь,PM-04,высокая,4,4,Печатник,None
16,2,Ночь,SM-01,струйная,1,5,Оператор струйной печати,None


None -> Нет кандидатов для index=2, плоская, min_rank=5
None -> Нет кандидатов для index=3, плоская, min_rank=4
None -> Нет кандидатов для index=4, плоская, min_rank=7
None -> Нет кандидатов для index=6, плоская, min_rank=5
None -> Нет кандидатов для index=7, плоская, min_rank=4
None -> Нет кандидатов для index=10, высокая, min_rank=5
None -> Нет кандидатов для index=11, высокая, min_rank=4
None -> Нет кандидатов для index=14, высокая, min_rank=5
None -> Нет кандидатов для index=15, высокая, min_rank=4
None -> Нет кандидатов для index=16, струйная, min_rank=5
None -> Нет кандидатов для index=17, струйная, min_rank=5
None -> Нет кандидатов для index=18, струйная, min_rank=5
None -> Нет кандидатов для index=19, струйная, min_rank=5
Остались свободные позиции оборудования после тура 6 назначения:


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None
6,2,Ночь,PM-02,плоская,3,5,Печатник,None
7,2,Ночь,PM-02,плоская,4,4,Печатник,None
10,2,Ночь,PM-03,высокая,3,5,Печатник,None
11,2,Ночь,PM-03,высокая,4,4,Печатник,None
14,2,Ночь,PM-04,высокая,3,5,Печатник,None
15,2,Ночь,PM-04,высокая,4,4,Печатник,None
16,2,Ночь,SM-01,струйная,1,5,Оператор струйной печати,None


In [543]:
summary = (
    shift_equipment_night
    .groupby(["machine_id", "machine_type"], as_index=False)
    .agg(
        required=("position", "count"),
        assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum())
    )
)

# Добавляем столбец "недостает"
summary["missing"] = summary["required"] - summary["assigned"]

# ---- 1. НЕПОЛНЫЕ БРИГАДЫ (частично укомплектованы)
incomplete = summary[
    (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
].copy()

incomplete

,machine_id,machine_type,required,assigned,missing
0,PM-01,плоская,4,2,2
1,PM-02,плоская,4,1,3
2,PM-03,высокая,4,2,2
3,PM-04,высокая,4,2,2


In [557]:
print(
    f"Назначено позиций: {shift_equipment_night['worker_id'].notna().sum()} из {len(shift_equipment_day)}"
)
if not free_machin_positions.empty:
    print("Не удалось назначить (после двух проходов):")
    display(
        free_machin_positions[["machine_id", "position", "machine_type", "min_rank"]]
    )

print(len(appointed))

Назначено позиций: 7 из 37
60


In [558]:
# Выводим свободные кандидаты после назначения
free_shift_workers = shift_workers_day[~shift_workers_day["worker_id"].isin(appointed)]
free_shift_workers

,week,shift,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии


In [559]:
# Выводим результат с именами работников
result = shift_equipment_day.merge(
    workers[["worker_id", "name"]], on="worker_id", how="left"
)
result

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id,name
0,2,День,PM-01,плоская,1,7,Печатник,W021,Работник П 21
1,2,День,PM-01,плоская,2,6,Печатник,W022,Работник П 22
2,2,День,PM-01,плоская,3,5,Печатник,W023,Работник П 23
3,2,День,PM-01,плоская,4,4,Печатник,W051,Работник В 51
4,2,День,PM-02,плоская,1,7,Печатник,W025,Работник П 25
5,2,День,PM-02,плоская,2,6,Печатник,W026,Работник П 26
6,2,День,PM-02,плоская,3,5,Печатник,W027,Работник П 27
7,2,День,PM-02,плоская,4,4,Печатник,W055,Работник В 55
8,2,День,PM-03,высокая,1,7,Печатник,W029,Работник В 29
9,2,День,PM-03,высокая,2,6,Печатник,W030,Работник В 30


In [551]:
# Группируем по машинам и позициям, чтобы получить минимальный разряд и требуемые профессии
by_machine_position = result.groupby(
    ["week", "shift", "machine_type", "machine_id", "position"], as_index=True
).agg(
    # machine_type=('machine_type', 'first'),
    # position = ('position', 'first'),
    min_rank=("min_rank", "min"),
    worker_id=("worker_id", "first"),
    name=("name", "first"),
    profession_required=("profession_required", "first"),
)

display(by_machine_position)

min_rank worker_id  \
week shift machine_type machine_id position                       
2    День  высокая      PM-03      1                7      W029   
                                   2                6      W030   
                                   3                5      W031   
                                   4                4      W043   
                        PM-04      1                7      W033   
                                   2                6      W034   
                                   3                5      W035   
                                   4                4      W047   
           плоская      PM-01      1                7      W021   
                                   2                6      W022   
                                   3                5      W023   
                                   4                4      W051   
                        PM-02      1                7      W025   
                                   2                6      W026   
                                   3                5      W027   
                                   4                4      W055   
           струйная     SM-01      1                5      W024   
                        SM-02      1                5      W028   
                        SM-03      1                5      W032   
                        SM-04      1                5      W036   
                        SM-05      1                5      W037   
                        SM-06      1                5      W038   
                        SM-07      1                5      W039   
                        SM-08      1                5      W040   
                        SM-09      1                5      W044   
                        SM-10      1                5      W048   
                        SM-11      1                5      W052   
                        SM-12      1                5      W056   
                        SM-13      1                5      W057   
                        SM-14      1                5      W058   
                        SM-15      1                5      W059   
                        SM-16      1                5      W060   
                        SM-17      1                5      W004   
                        SM-18      1                5      W008   
                        SM-19      1                5      W012   
                        SM-20      1                5      W016   
                        SM-21      1                5      W001   

                                                      name  \
week shift machine_type machine_id position                  
2    День  высокая      PM-03      1         Работник В 29   
                                   2         Работник В 30   
                                   3         Работник В 31   
                                   4         Работник П 43   
                        PM-04      1         Работник В 33   
                                   2         Работник В 34   
                                   3         Работник В 35   
                                   4         Работник П 47   
           плоская      PM-01      1         Работник П 21   
                                   2         Работник П 22   
                                   3         Работник П 23   
                                   4         Работник В 51   
                        PM-02      1         Работник П 25   
                                   2         Работник П 26   
                                   3         Работник П 27   
                                   4         Работник В 55   
           струйная     SM-01      1         Работник П 24   
                        SM-02      1         Работник П 28   
                        SM-03      1         Работник В 32   
                        SM-04      1         Работник В 36   
                        SM-05      1         Работник О 37   
     

In [552]:
by_machine_position = by_machine_position.reset_index()
by_machine_position

,week,shift,machine_type,machine_id,position,min_rank,worker_id,name,profession_required
0,2,День,высокая,PM-03,1,7,W029,Работник В 29,Печатник
1,2,День,высокая,PM-03,2,6,W030,Работник В 30,Печатник
2,2,День,высокая,PM-03,3,5,W031,Работник В 31,Печатник
3,2,День,высокая,PM-03,4,4,W043,Работник П 43,Печатник
4,2,День,высокая,PM-04,1,7,W033,Работник В 33,Печатник
5,2,День,высокая,PM-04,2,6,W034,Работник В 34,Печатник
6,2,День,высокая,PM-04,3,5,W035,Работник В 35,Печатник
7,2,День,высокая,PM-04,4,4,W047,Работник П 47,Печатник
8,2,День,плоская,PM-01,1,7,W021,Работник П 21,Печатник
9,2,День,плоская,PM-01,2,6,W022,Работник П 22,Печатник


In [553]:
schedule.head()

,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


In [554]:
temp = result[["week", "shift", "worker_id"]]
temp

,week,shift,worker_id
0,2,День,W021
1,2,День,W022
2,2,День,W023
3,2,День,W051
4,2,День,W025
5,2,День,W026
6,2,День,W027
7,2,День,W055
8,2,День,W029
9,2,День,W030
